# ETL Project 

Extraction: We obtained both of our data from Kaggle.com
    
    1st Data Set format: CSV
    2nd Data Set format: CSV

Transformation:
    
    1st Data Set: 
        1. took out un-needed columns 
        2. parse the Borough name out of the Jurisdiction Name field 
        3. GROUPBY the Borough to get total participates and total recieve public assistance
        4. we calculated the percent of people on public assistance and formatted into a percentage
        5. create a new dataframe with a reset index
    2nd Data Set:
        1. took out un-needed columns

Load:
   
   We loaded the data into 2 different MySQL tables because we knew that kind of data we where dealing with and it was well        structured. We merged the tables on the common column which was Borough.
    
    

Import need dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Load hood breakdown data into pandas
csv_file = "neighborhood-development-area-breakdowns.csv"
development_df = pd.read_csv(csv_file)
development_df.head()

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,Bronx Neighborhood Development Area 001,59,31,0.53,28,0.47,0,0,59,100,...,59,100,20,0.34,39,0.66,0,0,59,100
1,Bronx Neighborhood Development Area 002,7,5,0.71,2,0.29,0,0,7,100,...,7,100,3,0.43,4,0.57,0,0,7,100
2,Bronx Neighborhood Development Area 003,15,12,0.80,3,0.20,0,0,15,100,...,15,100,6,0.40,9,0.60,0,0,15,100
3,Bronx Neighborhood Development Area 004,44,27,0.61,17,0.39,0,0,44,100,...,44,100,13,0.30,31,0.70,0,0,44,100
4,Bronx Neighborhood Development Area 005,25,9,0.36,16,0.64,0,0,25,100,...,25,100,12,0.48,13,0.52,0,0,25,100


In [3]:
#Load crime into pandas
csv2_file = "NYPD_Complaint_Data_Historic.csv"
nypd_crime_df = pd.read_csv(csv2_file, dtype={'PREM_TYP_DESC':'object'})
nypd_crime_df.head()


C:\Users\Marc\Anaconda3\envs\PythonWebMongo\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,101109527,12/31/2015,23:45:00,NaN,NaN,12/31/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",...,44.0,INSIDE,BAR/NIGHT CLUB,NaN,NaN,1007314.0,241257.0,40.828848,-73.916661,"(40.828848333, -73.916661142)"
1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,103.0,OUTSIDE,NaN,NaN,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)"
2,569369778,12/31/2015,23:30:00,NaN,NaN,12/31/2015,117,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",...,28.0,NaN,OTHER,NaN,NaN,999463.0,231690.0,40.802607,-73.945052,"(40.802606608, -73.945051911)"
3,968417082,12/31/2015,23:30:00,NaN,NaN,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,105.0,INSIDE,RESIDENCE-HOUSE,NaN,NaN,1060183.0,177862.0,40.654549,-73.726339,"(40.654549444, -73.726338791)"
4,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,13.0,FRONT OF,OTHER,NaN,NaN,987606.0,208148.0,40.738002,-73.987891,"(40.7380024, -73.98789129)"


In [4]:
#Transform crime data
new_nypd_crime_df = nypd_crime_df[['BORO_NM','LAW_CAT_CD','OFNS_DESC']].copy()
new_nypd_crime_df.head()

,BORO_NM,LAW_CAT_CD,OFNS_DESC
0,BRONX,FELONY,FORGERY
1,QUEENS,FELONY,MURDER & NON-NEGL. MANSLAUGHTER
2,MANHATTAN,FELONY,DANGEROUS DRUGS
3,QUEENS,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES
4,MANHATTAN,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES


In [20]:
#Transform hood data
hood_df = development_df[['JURISDICTION NAME','COUNT PARTICIPANTS','COUNT RECEIVES PUBLIC ASSISTANCE']].copy()
hood_df.head()

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT RECEIVES PUBLIC ASSISTANCE
0,Bronx Neighborhood Development Area 001,59,20
1,Bronx Neighborhood Development Area 002,7,3
2,Bronx Neighborhood Development Area 003,15,6
3,Bronx Neighborhood Development Area 004,44,13
4,Bronx Neighborhood Development Area 005,25,12


In [6]:
# # new data frame with split value columns
new = hood_df["JURISDICTION NAME"].str.split(" ", n = 1, expand = True)

# # making separate first name column from new data frame
hood_df["BOROUGH"]= new[0]


# # Dropping old Name columns
hood_df.drop(columns =["JURISDICTION NAME"], inplace = True)

# df display
hood_df.head()

,COUNT PARTICIPANTS,COUNT RECEIVES PUBLIC ASSISTANCE,BOROUGH
0,59,20,Bronx
1,7,3,Bronx
2,15,6,Bronx
3,44,13,Bronx
4,25,12,Bronx


In [7]:
#Create new counts with sum and percent for Public Assistance
hood_group = hood_df.groupby(["BOROUGH"])
group_hood = hood_group.count()
group_hood["Total Participants"] = hood_group["COUNT PARTICIPANTS"].sum()
group_hood["Total Public Assistance"] = hood_group["COUNT RECEIVES PUBLIC ASSISTANCE"].sum()
group_hood["Percentage Recieves Public Assistance"] = group_hood["Total Public Assistance"] / group_hood["Total Participants"]
group_hood.head()

,COUNT PARTICIPANTS,COUNT RECEIVES PUBLIC ASSISTANCE,Total Participants,Total Public Assistance,Percentage Recieves Public Assistance
BOROUGH,,,,,
Bronx,12,12,556,187,0.336331
Brooklyn,17,17,1067,360,0.337395
Manhattan,6,6,127,30,0.236220
Not,1,1,2338,824,0.352438
Queens,7,7,48,5,0.104167


In [8]:
#Rename, drop and format columns 
group_hood = group_hood.rename(columns={'Total Participants': 'Total_Participants', 'Total Public Assistance': 'Total_Public_Assist', 'Percentage Recieves Public Assistance': 'Precent_Pub_Assist'})
group_hood = group_hood.drop(columns={'COUNT PARTICIPANTS', 'COUNT RECEIVES PUBLIC ASSISTANCE'})  
group_hood['Precent_Pub_Assist'] = group_hood['Precent_Pub_Assist']*100
group_hood['Precent_Pub_Assist'] = group_hood['Precent_Pub_Assist'].map("{:.3f}".format)


,Total_Participants,Total_Public_Assist,Precent_Pub_Assist
BOROUGH,,,
Bronx,556,187,33.633
Brooklyn,1067,360,33.739
Manhattan,127,30,23.622
Not,2338,824,35.244
Queens,48,5,10.417


In [17]:
#create new dataframe with reset index
new_hood = group_hood.reset_index()                  

In [23]:
#Connect to local database
rds_connection_string = "root:C@mryn2010@127.0.0.1/ny_crime"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [10]:
# Check for tables
engine.table_names()

['crime_data', 'neighborhoods']

In [18]:
#Use pandas to load csv converted DataFrame into database
new_hood.to_sql(name='neighborhoods', con=engine, if_exists='append', index=False)

In [12]:
#Use pandas to load csv converted DataFrame into database
new_nypd_crime_df.to_sql(name='crime_data', con=engine, if_exists='append', index=False)

In [21]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from neighborhoods', con=engine).head()

,id,BOROUGH,Total_Participants,Total_Public_Assist,Precent_Pub_Assist
0,26,Bronx,556,187,34.0
1,27,Brooklyn,1067,360,34.0
2,28,Manhattan,127,30,24.0
3,29,Not,2338,824,35.0
4,30,Queens,48,5,10.0


In [22]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from crime_data', con=engine).head()

,id,BORO_NM,LAW_CAT_CD,OFNS_DESC
0,1,BRONX,FELONY,FORGERY
1,2,QUEENS,FELONY,MURDER & NON-NEGL. MANSLAUGHTER
2,3,MANHATTAN,FELONY,DANGEROUS DRUGS
3,4,QUEENS,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES
4,5,MANHATTAN,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES


# Our MySQL code

Create our database

`CREATE DATABASE ny_crime;`

Use our database

`USE ny_crime;`

Create neighborhoods table with columns and defining columns types

`CREATE TABLE neighborhoods (
	id INT AUTO_INCREMENT NOT NULL,
    BOROUGH TEXT,
    Total_Participants INT,
    Total_Public_Assist INT,
    Precent_Pub_Assist DECIMAL,
    PRIMARY KEY (id)
    );`

Create neighborhoods table with columns and defining columns types

`CREATE TABLE crime_data (
	id INT AUTO_INCREMENT NOT NULL,
    BORO_NM TEXT,
    LAW_CAT_CD TEXT,
    OFNS_DESC TEXT,
    PRIMARY KEY (id)
    );`
   
Merge our two tables on Borough Name and select all needed columns  

`SELECT c.BORO_NM, c.LAW_CAT_CD, c.OFNS_DESC, n.Total_Participants, n.Total_Public_Assist, n.Precent_Pub_Assist
FROM crime_data AS c
INNER JOIN neighborhoods AS n
ON n.BOROUGH = c.BORO_NM;`
